In [1]:
%pip install wandb
%pip install ultralytics
%pip install matplotlib
%pip install numpy
%pip install opencv-python
%pip install inference

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb

from wandb.integration.ultralytics \
    import (
        add_wandb_callback
    )

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [3]:
from ultralytics    \
    import YOLO

from math           \
    import (
        floor
)

from random         \
    import SystemRandom

from os.path        \
    import (
        join
)

In [4]:
from os \
    import environ

environ['WANDB_DIR'] = '~/wandb'

environ['WANDB_CONSOLE']       = 'off'
environ['WANDB_DISABLE_CODE']  = 'on'
environ['WANDB_NOTEBOOK_NAME'] = 'TrainingOfModel.ipynb'
environ['WANDB_ENTITY']        = 'Designermadsen'
environ['WANDB_SILENT']        = 'True'

In [5]:
ZERO: int = 0
ONE: int = 1

In [6]:
EPOCHS: int = 200
BATCH: int  = 32
PATIENCE: int = 100

IMAGE_WIDTH_SIZE: int = 320
IMAGE_HEIGHT_SIZE: int = 320

IMAGE_DIMENSIONS: tuple = (
    IMAGE_HEIGHT_SIZE, 
    IMAGE_WIDTH_SIZE
)

SEED = None

IS_TO_RESUME: bool = False

MOSAIC_END_AT: int = -ONE

NUMBER_OF_TIMES_TO_ITERATE_TRAINING: int = 1

DEVICES_TO_USE: list = [
    ZERO
]

In [7]:
MODEL_WEIGHT_LOCATION: str | None = None
MODEL_ITERATION_VERSION: int = 10

FINISH_WHEN_DONE: bool = False

In [8]:
def get_weight_location() -> str:
    global MODEL_WEIGHT_LOCATION
    return MODEL_WEIGHT_LOCATION

def set_weight_location(
    value: str
) -> None:
    global MODEL_WEIGHT_LOCATION
    MODEL_WEIGHT_LOCATION = value

def get_current_model_iteration() -> int:
    global MODEL_ITERATION_VERSION
    return MODEL_ITERATION_VERSION

def set_current_model_iteration(
    value: int
) -> None:
    global MODEL_ITERATION_VERSION
    MODEL_ITERATION_VERSION = value

def increment_of_model() -> None:
    global ONE

    set_current_model_iteration(
        get_current_model_iteration() + ONE
    )

def generate_weight_location() -> str:
    weight_path = join(
        str(
            './runs/detect/train' 
        )        
        + 
        str(
            get_current_model_iteration()
        ),
        'weights/best.pt'
    )

    set_weight_location(
        weight_path
    )

    return get_weight_location()

In [9]:
HOME_LOCATION: str = '/home/ai'
DATASET_LOCATION:   str = join(
    HOME_LOCATION,
    'dataset.yaml'
)

YOLO_MODEL_VERSION: str = 'yolo11m.pt'

MODEL_WEIGHT_LOCATION = generate_weight_location()

In [10]:
generator = None
model = None

In [11]:
import logging
logger = logging.getLogger(
    __name__
)

logger.setLevel(
    logging.ERROR
)

In [12]:
PROJECT_NAME: str = 'spectre'

wandb.init(
    project     = PROJECT_NAME, 
    job_type    = 'training',
    save_code   = False
)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find TrainingOfModel.ipynb.
git root error: Cmd('git') failed due to: exit code(128)
  cmdline: git rev-parse --show-toplevel
  stderr: 'fatal: detected dubious ownership in repository at '/mnt/c/Users/Kentv/Desktop/Spectre/Repository'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/c/Users/Kentv/Desktop/Spectre/Repository'


In [ ]:
model = YOLO(
    YOLO_MODEL_VERSION,
    verbose = False
)

add_wandb_callback(
    model,
    enable_train_validation_logging     = False,
    enable_validation_logging           = False,
    enable_prediction_logging           = False,
    enable_model_checkpointing          = False
)

if not (
    MODEL_WEIGHT_LOCATION is None
):
    model.load(
        get_weight_location()
    )

Transferred 643/649 items from pretrained weights


In [15]:
def getSeed() -> int:
    global SEED
    
    if isSeedNone():
        return generateSeed()

    return SEED

def  setSeed(
    value: int
) -> None:
    global SEED
    SEED = value


def generateGenerator() -> None:
    global generator

    if generator is None:
        generator = SystemRandom()

def isSeedNone() -> bool:
    global SEED
    return SEED is None

def generateSeed() -> None:
    global          \
        SEED,       \
        generator,  \
        ZERO
    
    generateGenerator()

    setSeed(
        generator.randint(
            ZERO, 
            2147483647
        )
    )

    return SEED

In [ ]:
if not IS_TO_RESUME:
    generateSeed()

for idx in range(
    NUMBER_OF_TIMES_TO_ITERATE_TRAINING
):
    training_results =                          \
        model.train(
            data         = DATASET_LOCATION, 
            epochs       = EPOCHS,
            batch        = BATCH, 
            imgsz        = IMAGE_DIMENSIONS,
            device       = DEVICES_TO_USE,
            plots        = True,
            patience     = PATIENCE,
            close_mosaic = MOSAIC_END_AT,
            seed         = getSeed(),
            multi_scale  = False,
            resume       = IS_TO_RESUME,
            save         = True
    )
    
    if not (
        int( 
            NUMBER_OF_TIMES_TO_ITERATE_TRAINING 
            - 
            1 
        )
        == 
        idx
    ):
        increment_of_model()

        model.load(
            generate_weight_location()
        )


New https://pypi.org/project/ultralytics/8.3.73 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.72 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/home/ai/dataset.yaml, epochs=200, time=None, patience=100, batch=32, imgsz=(320, 320), save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=False, pretrained=./runs/detect/train10/weights/best.pt, optimizer=auto, verbose=True, seed=1895186311, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=-1, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=

train: Scanning /opt/dataset/tmod/training/labels... 3600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:03<00:00, 929.20it/s]

train: New cache created: /opt/dataset/tmod/training/labels.cache



val: Scanning /opt/dataset/tmod/validation/labels... 900 images, 0 backgrounds, 0 corrupt: 100%|██████████| 900/900 [00:01<00:00, 728.19it/s]

val: New cache created: /opt/dataset/tmod/validation/labels.cache


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      4.62G     0.5098     0.8056     0.9625         55        320: 100%|██████████| 113/113 [00:35<00:00,  3.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:07<00:00,  2.13it/s]

                   all        900       1087      0.977      0.826      0.868      0.746


In [ ]:
if FINISH_WHEN_DONE:
    wandb.finish()